In [ ]:
!pip install category_encoders
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
!pip install feature-engine
!pip install -U imbalanced-learn
!pip install pdpbox
!pip install shap

In [ ]:
#from pandas_profiling import ProfileReport
#ProfileReport(X)

In [ ]:
import pandas as pd
import numpy as np

from feature_engine.imputation import RandomSampleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,SimpleImputer
from sklearn.ensemble import RandomForestRegressor


train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dacon_credit/train.csv',index_col='index')

def feature_eng(df):
    #gender
    df.gender = df.gender.replace('F',0) #여성 0
    df.gender = df.gender.replace('M',1) #남성 1

    #car
    df.car = df.car.replace('N',0)
    df.car = df.car.replace('Y',1)

    #reality
    df.reality = df.reality.replace('N',0)
    df.reality = df.reality.replace('Y',1)
    
    #income:  log_transform
    transformer = FunctionTransformer(np.log1p, validate=True)
    df.income_total = transformer.transform([df.income_total])[0]
    
    # to abs
    df.DAYS_BIRTH = abs(df.DAYS_BIRTH)
    df.DAYS_EMPLOYED = abs(df.DAYS_EMPLOYED)
    
    # outlier , randomforesst impute
    df['DAYS_EMPLOYED'].loc[df['DAYS_EMPLOYED']>300000] = None
    rf = RandomForestRegressor()
    impute = IterativeImputer(random_state=42, estimator =rf)
    reshape = np.array(df['DAYS_EMPLOYED']).reshape(-1,1)
    df['DAYS_EMPLOYED'] = impute.fit_transform(reshape)


    rs_impute = RandomSampleImputer(random_state=42)
    s_impute = SimpleImputer(strategy='most_frequent')
    #df['occyp_type'] = rs_impute.fit_transform(df[['occyp_type']])
    df['occyp_type'] = s_impute.fit_transform(df[['occyp_type']])


    
    #df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].fillna(df['DAYS_EMPLOYED'].mean())
    
    #all ones
    df = df.drop(columns= 'FLAG_MOBIL')
    
    #new feature
    df['DAYS_UNEMPLOYED'] = df.DAYS_BIRTH  -  df.DAYS_EMPLOYED
    
    df['in_out_ratio'] = (df['reality']*100 + df['car']*10 + df['child_num'])/(1+df['DAYS_EMPLOYED']*df['begin_month']*df['income_total']/1000)

    #중복 데이터들 어떻게 처리할까..포기!
    df['card_num'] = 1
    key_columns = df.drop(columns = ['begin_month','card_num']).columns

    condition1 = df.duplicated(subset = key_columns,
                               keep = False) #2개이상
    condition2 = df.loc[condition1].duplicated(subset = key_columns,
                                               keep = False)#3개이상
    condition3 = df.loc[condition1].loc[condition2].duplicated(subset = key_columns,
                                                               keep = False)#4개이상
    condition4 = df.loc[condition1].loc[condition2].loc[condition3].duplicated(subset = key_columns,
                                                                               keep = False)

    index = df['card_num'].loc[(condition1)& ~(condition2)].index 
    df['card_num'].loc[index]= 2
    index = df['card_num'].loc[condition1].loc[(condition2)& ~(condition3)].index
    df['card_num'].loc[index]= 3
    index = df['card_num'].loc[condition1].loc[condition2].loc[(condition3)&~(condition4)].index
    df['card_num'].loc[index]= 4
    df = df.drop(columns = 'card_num')
    df = df.drop(columns = 'email')



    return df

#One_Hot
#income_type

target = 'credit'

X = feature_eng(train)
y = X[target]
X = X.drop(columns = target)

X_train,X_val,y_train,y_val = train_test_split(X,y, test_size = 0.2)

In [ ]:
X.head().T

In [ ]:
'''
from imblearn.over_sampling import RandomOverSampler,SMOTE,ADASYN,SMOTEN
from imblearn.under_sampling import TomekLinks,EditedNearestNeighbours,InstanceHardnessThreshold,NeighbourhoodCleaningRule
from sklearn.linear_model import LogisticRegression

ros = NeighbourhoodCleaningRule()
enc = TargetEncoder()
Z_train = enc.fit_transform(X_train,y_train)
Z_val = enc.transform(X_val,y_val)

X_train_o,y_train_o = ros.fit_resample(Z_train,y_train)
'''
#샘플링 의미가 별로 없었다.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

#baseline

baseline = y_train.value_counts(normalize= True)
y_pred = [baseline]*len(X_train)
y_pred_val = [baseline]*len(X_val)

print('훈련 log_loss:', log_loss(y_train,y_pred))
print('검증 log_loss:', log_loss(y_val,y_pred_val))

In [ ]:
from xgboost import XGBClassifier,DMatrix
from category_encoders import OrdinalEncoder
from sklearn.utils import class_weight

#xgboost
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

pipe = make_pipeline(TargetEncoder(),
                     XGBClassifier(random_state=42,
                                   objective = 'multi:softprob',
                                   class_weight = classes_weights,
                                   xgbclassifier__colsample_bytree = 0.6,
                                   gamma=0.3,
                                   max_depth = 9,
                                   min_child_weight = 2,
                                   reg_alpha = 1,
                                   sub_sample = 0.8,
                                   learning_rate = 0.1
                                   ))

#xgboost.DMatrix(..., weight = *weight array for individual weights*)

{'xgbclassifier__colsample_bytree': [0.5, 0.6, 0.7],
  'xgbclassifier__gamma': [0.2,0.3,0.4],
  'xgbclassifier__max_depth': [8,9,10],
  'xgbclassifier__min_child_weight': [1,2,3],
  'xgbclassifier__reg_alpha': [0.1,1,10],
  'xgbclassifier__subsample': [0.7,0.8,0.9]
  }




pipe.fit(X_train,y_train)
y_pred = pipe.predict_proba(X_train)
y_pred_val = pipe.predict_proba(X_val)

print('훈련 log_loss:', log_loss(y_train,y_pred))
print('검증 log_loss:', log_loss(y_val,y_pred_val))

In [ ]:
import matplotlib.pyplot as plt
clf = pipe.named_steps['xgbclassifier']
importanc = clf.feature_importances_
columns = X_train.columns

plt.barh(columns, importanc)

In [ ]:
#xgboost random search
rsc1_param = {
    'xgbclassifier__max_depth':range(3,10),
    'xgbclassifier__min_child_weight':range(1,6),
    'xgbclassifier__gamma':[i/10.0 for i in range(0,5)],
    'xgbclassifier__subsample':[i/10.0 for i in range(6,10)],
    'xgbclassifier__colsample_bytree':[i/10.0 for i in range(6,10)],
    'xgbclassifier__reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]

}
rsc1 = RandomizedSearchCV(estimator=pipe,
                          param_distributions = rsc1_param,
                          n_iter = 50,
                          scoring = 'neg_log_loss',
                          cv = 3,
                          random_state= 42)
rsc1.fit(X_train,y_train)
rsc1.cv_results_, rsc1.best_params_, rsc1.best_score_

In [ ]:
#xgboost grid search

param_test=  {'xgbclassifier__colsample_bytree': [0.5, 0.6, 0.7],
  'xgbclassifier__gamma': [0.2,0.3,0.4],
  'xgbclassifier__max_depth': [8,9,10],
  'xgbclassifier__min_child_weight': [1,2,3],
  'xgbclassifier__reg_alpha': [0.1,1,10],
  'xgbclassifier__subsample': [0.7,0.8,0.9]
  }



xgb_grid = GridSearchCV(estimator= pipe,
                        param_grid = param_test,
                        scoring='neg_log_loss', n_jobs=-1, cv=5)
xgb_grid.fit(X_train,y_train)
xgb_grid.cv_results_, xgb_grid.best_params_, xgb_grid.best_score_

In [ ]:
 {'randomforestclassifier__bootstrap': False,
  'randomforestclassifier__max_depth': 40,
  'randomforestclassifier__max_features': 'auto',
  'randomforestclassifier__min_samples_leaf': 4,
  'randomforestclassifier__min_samples_split': 10,
  'randomforestclassifier__n_estimators': 400},

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#RandomForest
pipe2 = make_pipeline(TargetEncoder(),
                      RandomForestClassifier(random_state=15,
                                             bootstrap = False,
                                             max_depth = 40,
                                             max_features = 'auto',
                                             min_samples_leaf = 4,
                                             min_samples_split = 10,
                                             n_estimators = 400
                                             ))
pipe2.fit(X_train,y_train)
y_pred = pipe2.predict_proba(X_train)
y_pred_val = pipe2.predict_proba(X_val)

print('훈련 log_loss:', log_loss(y_train,y_pred))
print('검증 log_loss:', log_loss(y_val,y_pred_val))

In [ ]:
#random search

random_param ={
    'randomforestclassifier__bootstrap': [True, False],
    'randomforestclassifier__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'randomforestclassifier__max_features': ['auto', 'sqrt'],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__n_estimators': [200, 400]
    }

rsc = RandomizedSearchCV(pipe2,
                         param_distributions = random_param,
                         n_iter = 50,
                         scoring = 'neg_log_loss',
                         cv = 4,
                         random_state = 42)

rsc.fit(X_train,y_train)

In [ ]:
rsc.cv_results_, rsc.best_params_, rsc.best_score_

In [ ]:
clf = pipe2.named_steps['randomforestclassifier']
importanc = clf.feature_importances_
columns = X_train.columns
plt.barh(columns, importanc)

In [ ]:
#dacon 제출용
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dacon_credit/test.csv',index_col= 'index')
X_test = feature_eng(test)
pipe.fit(X,y)
y_pred_test = pipe2.predict_proba(X_test)
submission = pd.DataFrame(y_pred_test)

index = pd.DataFrame(X_test.index)
submission = pd.concat([index,submission], axis =1)
submission.columns = ['index',0,1,2]
submission.to_csv('submission.csv',index = False)

###시각화

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot


encoder = TargetEncoder()
X_train_encoded = encoder.fit_transform(X_train,y_train) # 학습데이터
X_val_encoded = encoder.transform(X_val,y_val) # 검증데이터

In [ ]:
rf = pipe2.named_steps['randomforestclassifier']

feature = 'begin_month'
isolated = pdp_isolate(
    model=rf, 
    dataset=X_val_encoded, 
    model_features=X_val_encoded.columns, 
    feature=feature,
    num_grid_points = 50
)

pdp_plot(isolated, feature_name=feature);

In [ ]:
feature = 'income_total'

isolated = pdp_isolate(
    model=rf, 
    dataset=X_val_encoded, 
    model_features=X_val_encoded.columns, 
    feature=feature,
    num_grid_points = 100
)

pdp_plot(isolated, 
         feature_name=feature)

In [ ]:
feature = 'DAYS_BIRTH'

isolated = pdp_isolate(
    model=rf, 
    dataset=X_val_encoded, 
    model_features=X_val_encoded.columns, 
    feature=feature,
    num_grid_points = 50
)

pdp_plot(isolated, feature_name=feature);

In [ ]:
import shap

row = X_val_encoded.iloc[2]

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(row)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value[2], 
    shap_values=shap_values[2],
    features=row
)

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(X_val_encoded.iloc[:50])
shap.force_plot(explainer.expected_value[0], shap_values[0], X_val_encoded.iloc[:50])

In [ ]:
21000/365

begin month 초반에는 좋은 신용도를 보여주는데 처음에는 처음이다보니 잘 상환해야겠다는 마음이 들지만 이후부터는 느슨해지면서 크게 영향을 받지 않는다.(사바사다)
total income 일정 수준 이상의 수입에서는 높은 신용도에 긍정적인 영향을 준다.
child_num 많을 수록 신용도가 떨어지는데 아마 예상치 못한 지출이 늘어나는 경향이 있는 것이 아닐까라는 생각이 든다
family size 마찬가지로 자녀 숫자의 영향으로 그런것이 아닌가 생각됨
car 자동차가 있는 경우 생각보다 긍정적인 영향을 주었다. 일시불로 사지 않을 가능성이 크므로 차량 소유 여부가 부정적인 영향을 끼칠수 있다고 생각했는데, 그런 경우도 간혹 있지만
대부분 차량 관련 상환은 충실히 한다는 생각을 해볼 수 있지 않을까? 추가적인 조사가 필요하다. 

남성이 여성보다 긍정적인 영향이 있지만 큰 차이는 아님(여성도 무조건 +)

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(X_val_encoded.iloc[:50])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_val_encoded.iloc[:50])

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(X_val_encoded.iloc[:50])
shap.force_plot(explainer.expected_value[2], shap_values[2], X_val_encoded.iloc[:50])